In [4]:
def show_forecaster_details(client, fid: str):
    """Call the API and pretty-print forecaster details."""
    details = client.get_forecaster_details(fid)
    if not details:
        print("❌ No details available")
        return

    print("\n=== Forecaster Details ===")
    print(f"ID: {details.get('forecasterId', '-')}")
    print(f"Name: {details.get('forecasterName', '-')}")
    
    # Features
    features = details.get("features", [])
    print(f"Features: {', '.join(features) if features else '-'}")

    # Properties
    props = details.get("properities", {})
    if props:
        print("\n--- Properties ---")
        for key, value in props.items():
            print(f"{key}: {value}")
    else:
        print("No properties available")


In [6]:
def show_forecaster_data(client, fid: str):
        """Call the API and pretty-print all data currently held by a forecaster."""
        data = client.get_forecaster_data(fid)
        if not data or "data" not in data:
            print("❌ No data available for this forecaster")
            return

        print(f"\n=== Forecaster Data (total {len(data['data'])} points) ===")
        
        for ts, values in data["data"].items():
            features = ", ".join([f"{k}={v}" for k, v in values.items()])
            print(f"{ts} → {features}")


In [ ]:
from driftmind import DriftMindClient
from driftmind.generator import generate_sin_cos_tan_with_drifts
from driftmind import utils

import pandas as pd
import matplotlib.pyplot as plt

columns = ["Sin", "Cos", "Tan"]
results = {"Sin": [], "Cos": [], "Tan": []}

# 1 Get Credentials
creds = utils.load_credentials()
if creds:
    api_key = creds["DRIFTMIND_API_KEY"]
    base_url = creds["DRIFTMIND_API_URL"]
    print("API key loaded:", bool(api_key))
    print("Base URL:", base_url)
else:
    print("Failed to load DriftMind credentials.")
    exit(1)

# 2. Connect to DriftMind
client = DriftMindClient(api_key, base_url)

# 2. Create fresh forecaster (cold start)
fid = client.create_forecaster({
    "forecasterName": "Cold Start Demo",
    "features":columns,
    "inputSize": 15,
    "outputSize": 1
})["forecaster_id"]

print("Created new forecaster with ID:", fid)

# 3. Generate dataset with drifts
df = generate_sin_cos_tan_with_drifts(n=600)
#3.1 return forecasters details
show_forecaster_details(client, fid)

# 4. Online loop: feed one point → try forecast → store outside the package

print("Starting online learning and forecasting...")
global_results = {"timestamp": [], "anomalyScore": [], "numberOfClusters": []}
for _, row in df.iterrows():
    point = {}
    for col in columns:
        point[col] = [float(row[col])]
    client.feed_point(fid, point)

    yhat = client.forecast(fid)
    if yhat is not None:
        global_results["timestamp"].append(int(row["Sequence"]))
        global_results["anomalyScore"].append(float(yhat.get("anomalyScore", 0)))
        global_results["numberOfClusters"].append(int(yhat.get("numberOfClusters", 0)))
 
        for var in columns:
            feature = yhat.get("FeaturesMap", {}).get(var, {})
            pred = feature.get("predictions", [])
            if pred:
                results[var].append({
                    "expected": float(row[var]),
                    "predicted": float(pred[0]),
                    "timestamp": int(row["Sequence"])
                })
                print(
                    f"Fed Sequence={int(row['Sequence'])}, "
                    f"Expected {var}={float(row[var]):.3f}, "
                    f"Predicted {yhat.get('FeaturesMap', {}).get(var, {}).get('predictions', [0])[0]:.3f}, "
                    f"SME={abs(float(row[var]) - yhat.get('FeaturesMap', {}).get(var, {}).get('predictions', [0])[0]):.3f}"
)



# Visualize results
for var in columns:
    df_var = pd.DataFrame(results[var])
    utils.plot_actual_vs_predicted(df_var, var)

# Visualize global results
df_global = pd.DataFrame(global_results)

utils.plot_time_series(
    df_global["timestamp"],
    df_global["anomalyScore"],
    "Global Anomaly Score over Time",
    "Time Step",
    "Anomaly Score"
)

utils.plot_time_series(
    df_global["timestamp"],
    df_global["numberOfClusters"],
    "Global Number of Clusters over Time",
    "Time Step",
    "Number of Clusters"
)

# Show forecaster current forecasting data
show_forecaster_data(client, fid)
